## LangChain으로 Tool 사용하기 

### 1. 도구 정의 (Tool Definition)
- 핵심: `@tool` 데코레이터를 사용해 일반 파이썬 함수를 랭체인 도구로 변환합니다.
- Docstring의 역할: LLM은 파이썬 코드를 읽는 게 아니라 **함수 설명 Docstring**을 읽고 "이 도구가 무슨 일을 하는지" 판단합니다. 설명이 부실하면 LLM이 도구를 선택하지 못합니다.
- 타입 힌트: a: int와 같은 타입 지정은 LLM이 인자(args)를 어떤 자료형으로 넘겨줄지 결정하는 기준이 됩니다.

### 2. 도구 바인딩 (Binding Tools)
- 메서드: llm_with_tools = `llm.bind_tools([함수명])`
- 내부 동작: 랭체인은 우리가 정의한 함수를 JSON Schema 형태로 변환합니다. 이 스키마에는 함수의 이름, 설명, 필요한 파라미터 정보가 포함됩니다.
- 연결 의미: LLM 객체에게 "너는 대답하기 전에 이 도구 목록(JSON)을 참고해서, 필요하면 나에게 실행을 요청해!"라고 가이드를 전달하는 설정입니다.

### 3. 호출 및 판단 (Invoke & Reasoning)
- 과정: `llm_with_tools.invoke("질문")` 호출 시, 질문과 함께 **도구 명세서 JSON**가 모델 API로 전송됩니다.
- LLM의 판단: 모델은 질문을 분석한 뒤, 직접 답할지 아니면 도구를 쓸지 결정합니다.
- 반환값 (AI Message)
    - 도구를 사용하기로 결정한 경우: content는 비우고 tool_calls에 실행할 도구의 이름(name), 입력값(args), 고유 ID가 담긴 리스트를 채워서 반환함 
    - 도구를 사용하지 않고 직접 답하기로 결정한 경우: tool_calls에는 비우고, content에 답변을 직접 채워서 반환함 
- 반드시 도구를 하나라도 사용하도록 강제하는 설정도 존재함 
    - llm_forced_to_calc = small_llm.bind_tools([add, multiply], tool_choice="required")

### 4. 실행 및 최종 답변 (Execution & Final Response)
- 도구 실행: LLM이 제안한 tool_calls 정보를 바탕으로 실제 파이썬 함수를 실행합니다.
- ToolMessage 생성: 도구 실행 결과값(content)과 함께, 반드시 LLM이 부여했던 **고유 ID(tool_call_id)**를 매칭하여 메시지를 생성합니다.
- 컨텍스트 완성: [사용자 질문, LLM의 도구 호출 제안, 실제 도구 실행 결과] 세 가지 메시지를 리스트로 묶어 다시 LLM을 호출합니다.
- 최종 답변: 모든 정보를 전달받은 LLM은 비로소 tool_calls 대신 content에 최종 답변을 채워서 반환합니다.

In [ ]:
# TOOL 함수의 정보를 LLM에게 전달하는 도구 명세서 JSON 예시    
{
  "name": "add",
  "description": "숫자 a와 b를 더합니다.",  # <--- 바로 여기가 Docstring 내용!
  "parameters": {
    "type": "object",
    "properties": {
      "a": {"type": "integer"},
      "b": {"type": "integer"}
    },
    "required": ["a", "b"]
  }
}

In [28]:
from langchain_core.tools import tool

#이 함수가 도구(tool)라는 것을 알려주기 위한 어노테이션
@tool 
#@tool(args_schema=MyInputSchema, description="여기에 설명을 적어도 됩니다")
def add(a: int, b:int) -> int:
    # LangChain의 @tool 데코레이터는 함수에 docstring(함수에 대한 설명 주석)이 필요합니다. 
    # docstring이 없으면 description 파라미터를 제공해야 합니다.
    """숫자 a와 b를 더합니다.""" 
    return a + b

@tool
def multiply(a: int, b:int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [2]:
#tool로 정의한 함수 호출 방법 
#add(4, 8) #에러 
add.invoke({'a': 4, 'b': 8})

add 실행됩니다


12

In [3]:
from langchain_openai import ChatOpenAI

small_llm = ChatOpenAI(model='gpt-4o')
#small_llm = ChatOpenAI(model='gpt-4o-mini')


In [4]:
# bind_tools: "이 LLM 객체가 호출될 때, 사용할 수 있는 도구들의 명세서(Blueprint)를 함께 전달하도록 세팅하는 것"
# 동작 원리: small_llm.bind_tools([add, multiply])를 하면, 내부적으로는 우리가 만든 파이썬 함수를 JSON Schema라는 규격으로 변환합니다.
# 고정된 연결: 이제 llm_with_tools 객체를 사용할 때마다, 랭체인은 모델(GPT-4o-mini 등)에게 "너는 대답할 때 이 도구 명세서들을 참고해서, 필요하면 이 함수를 쓰겠다고 말해줘"라고 매번 알려주는 상태가 된 것입니다.
llm_with_tools = small_llm.bind_tools([add, multiply])

In [ ]:
query = "3 곱하기 5는?"
small_llm.invoke(query)

In [34]:
query = "3 곱하기 5는?" # tool_calls에 multiply 함수 채워서 반환  
# query = "미국의 수도는?" #답변을 content에 직접 반환 
ai_message = llm_with_tools.invoke(query)

In [35]:
ai_message

'''
AIMessage(
content='',
additional_kwargs={
  'refusal': None
},
response_metadata={
  'token_usage': {
    'completion_tokens': 17,
    'prompt_tokens': 84,
    'total_tokens': 101,
    'completion_tokens_details': {
      'accepted_prediction_tokens': 0,
      'audio_tokens': 0,
      'reasoning_tokens': 0,
      'rejected_prediction_tokens': 0
    },
    'prompt_tokens_details': {
      'audio_tokens': 0,
      'cached_tokens': 0
    }
  },
  'model_provider': 'openai',
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_084a28d6e8',
  'id': 'chatcmpl-DAthPuDctj6Ez2GrnMp2VVIsYCGNK',
  'service_tier': 'default',
  'finish_reason': 'tool_calls',
  'logprobs': None
},
id='lc_run--019c7504-8e08-75e2-b032-89488e048821-0',
tool_calls=[
  {
    'name': 'multiply',
    'args': {
      'a': 3,
      'b': 5
    },
    'id': 'call_PXganPzH2xzknY73JSiSRlBE',
    'type': 'tool_call'
  }
],
invalid_tool_calls=[
  
],
usage_metadata={
  'input_tokens': 84,
  'output_tokens': 17,
  'total_tokens': 101,
  'input_token_details': {
    'audio': 0,
    'cache_read': 0
  },
  'output_token_details': {
    'audio': 0,
    'reasoning': 0
  }
})
'''


"\nAIMessage(\ncontent='',\nadditional_kwargs={\n  'refusal': None\n},\nresponse_metadata={\n  'token_usage': {\n    'completion_tokens': 17,\n    'prompt_tokens': 84,\n    'total_tokens': 101,\n    'completion_tokens_details': {\n      'accepted_prediction_tokens': 0,\n      'audio_tokens': 0,\n      'reasoning_tokens': 0,\n      'rejected_prediction_tokens': 0\n    },\n    'prompt_tokens_details': {\n      'audio_tokens': 0,\n      'cached_tokens': 0\n    }\n  },\n  'model_provider': 'openai',\n  'model_name': 'gpt-4o-mini-2024-07-18',\n  'system_fingerprint': 'fp_084a28d6e8',\n  'id': 'chatcmpl-DAthPuDctj6Ez2GrnMp2VVIsYCGNK',\n  'service_tier': 'default',\n  'finish_reason': 'tool_calls',\n  'logprobs': None\n},\nid='lc_run--019c7504-8e08-75e2-b032-89488e048821-0',\ntool_calls=[\n  {\n    'name': 'multiply',\n    'args': {\n      'a': 3,\n      'b': 5\n    },\n    'id': 'call_PXganPzH2xzknY73JSiSRlBE',\n    'type': 'tool_call'\n  }\n],\ninvalid_tool_calls=[\n\n],\nusage_metadata={\n

In [38]:
ai_message.tool_calls # 리스트 

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_kFNflk7yQdIiTR5M56SpwN8K',
  'type': 'tool_call'}]

In [24]:
# 1. 메시지에서 도구 호출 정보 추출
tool_call = ai_message.tool_calls[0] # 첫 번째 제안 선택

{'name': 'add',
 'args': {'a': 3, 'b': 5},
 'id': 'call_PRApXsBflPoh38dNtuXQk55s',
 'type': 'tool_call'}

In [48]:
# 2. 직접 함수 실행 (이때 tool_call 정보를 통째로 넘김)
tool_message = multiply.invoke(tool_call) # args라는 키의 값에서 a와 b 값만 알아서 추출해서 실제 함수에 넣어줌 
tool_message # 결과는 ToolMessage 형태로 나옴

ToolMessage(content='15', name='multiply', tool_call_id='call_kFNflk7yQdIiTR5M56SpwN8K')

In [40]:
from typing import Sequence

from langchain_core.messages import AnyMessage, HumanMessage

human_message = HumanMessage(query) # "3 곱하기 5는?" 
human_message

HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={})

In [ ]:
# 3. 이 메시지를 다시 메시지 리스트에 수동으로 append
message_list: Sequence[AnyMessage] = [human_message] # 사용자 질문 

message_list.append(ai_message) # 사용할 tool 

message_list.append(tool_message) # tool을 사용해서 나온 결과 

In [42]:
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_64dfa806c7', 'id': 'chatcmpl-DCKZq9eGJ2ICllPBjZog0hBiHz8wD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8961-a9a4-7692-ac0d-39914b7bf287-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_kFNflk7yQdIiTR5M56SpwN8K', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 

In [ ]:
result = llm_with_tools.invoke(message_list) #모든 컨텍스트를 전달받은 LLM은 최종 답변을 content에 넣어 반환 

In [46]:
result 
'''
(content='3 곱하기 5는 15입니다.',
additional_kwargs={
  'refusal': None
},
response_metadata={
  'token_usage': {
    'completion_tokens': 12,
    'prompt_tokens': 109,
    'total_tokens': 121,
    'completion_tokens_details': {
      'accepted_prediction_tokens': 0,
      'audio_tokens': 0,
      'reasoning_tokens': 0,
      'rejected_prediction_tokens': 0
    },
    'prompt_tokens_details': {
      'audio_tokens': 0,
      'cached_tokens': 0
    }
  },
  'model_provider': 'openai',
  'model_name': 'gpt-4o-2024-08-06',
  'system_fingerprint': 'fp_01cbaa0587',
  'id': 'chatcmpl-DCKqm06SuZcZqxDFdCtJST12FlmHg',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None
},
id='lc_run--019c8971-b340-7653-bb09-82862f8cd26d-0',
tool_calls=[
  
],
invalid_tool_calls=[
  
],
usage_metadata={
  'input_tokens': 109,
  'output_tokens': 12,
  'total_tokens': 121,
  'input_token_details': {
    'audio': 0,
    'cache_read': 0
  },
  'output_token_details': {
    'audio': 0,
    'reasoning': 0
  }
})
'''

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 109, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_01cbaa0587', 'id': 'chatcmpl-DCKqm06SuZcZqxDFdCtJST12FlmHg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c8971-b340-7653-bb09-82862f8cd26d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 109, 'output_tokens': 12, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})